In [1]:
import glob
import numpy as np
from batter import MABFESystem
from batter.input_process import get_configure_from_file, parse_input_file

from logging import getLogger, DEBUG 
logger = getLogger()
logger.setLevel(DEBUG)

/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
protein_file = 'data/7T2G_mp/protein_input.pdb'
system_file = 'data/7T2G_mp/system_input.pdb'
ligand_files = ['data/7T2G_mp/ligand_MP.pdb',
                'data/7T2G_mp/ligand_MP2.pdb',
                'data/7T2G_mp/ligand_DAMGO.pdb']
#system_inpcrd = 'data/7T2G_mp/system_input.inpcrd'
equilibrated_rst = 'data/7T2G_mp/equilibrated.rst'

In [3]:
output_folder = 'test/abfe'
system = MABFESystem(folder=output_folder)

INFO | Creating a new system: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/abfe/


In [5]:
# check available input of create_system
system.create_system?

Signature:
test_system.create_system(
    system_name: str,
    protein_input: str,
    system_topology: str,
    ligand_paths: List[str],
    receptor_segment: str = None,
    system_coordinate: str = None,
    protein_align: str = 'name CA and resid 60 to 250',
    retain_lig_prot: bool = True,
    ligand_ph: float = 7.4,
    ligand_ff: str = 'gaff2',
    lipid_mol: List[str] = [],
    lipid_ff: str = 'lipid21',
    overwrite: bool = False,
)
Docstring:
Create a new single-ligand single-receptor system.

Parameters
----------
protein_input : str
    Path to the protein file in PDB format.
    It should be exported from Maestro,
    which means the protonation states of the protein are assigned.
    Water and ligand can be present in the file,
    but they will be removed during preparation.
system_topology : str
    PDB file of a prepared simulation system with `dabble`.
    The ligand does not need to be present.
system_coordinate : str
    The coordinate file for the system.
    Th

In [6]:
system.create_system(
            system_name='7T2G',
            protein_input=protein_file,
            system_topology=system_file,
            system_coordinate=equilibrated_rst,
            ligand_paths=ligand_files,
#            overwrite=True,
            lipid_mol=['POPC'])

INFO | self: <batter.batter.MABFESystem object at 0x7f2740fddb50>
INFO | system_name: 7T2G
INFO | protein_input: data/7T2G_mp/protein_input.pdb
INFO | system_topology: data/7T2G_mp/system_input.pdb
INFO | ligand_paths: ['data/7T2G_mp/ligand_MP.pdb', 'data/7T2G_mp/ligand_MP2.pdb', 'data/7T2G_mp/ligand_DAMGO.pdb']
INFO | receptor_segment: None
INFO | system_coordinate: data/7T2G_mp/equilibrated.rst
INFO | protein_align: name CA and resid 60 to 250
INFO | retain_lig_prot: True
INFO | ligand_ph: 7.4
INFO | ligand_ff: gaff2
INFO | lipid_mol: ['POPC']
INFO | lipid_ff: lipid21
INFO | overwrite: False
WARNING | Elongating the ligand name: mp to mp1
INFO | The net charge of the ligand mp1 in /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/abfe//ff/mp1.pdb is 1.0
WARNING | Elongating the ligand name: mp to mp2
INFO | The net charge of the ligand mp2 in /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/abfe//ff/mp2.pdb is 1.0
WARNING | Shortening 

In [7]:
import MDAnalysis as mda

u_prot = mda.Universe(f'{output_folder}/all-poses/reference.pdb')
u_lig = mda.Universe(f'{output_folder}/all-poses/pose0.pdb')
u_merge = mda.Merge(u_prot.atoms, u_lig.atoms)

P1_atom = u_merge.select_atoms('name CA and resid 149')
P2_atom = u_merge.select_atoms('name CA and resid 119')
P3_atom = u_merge.select_atoms('name CA and resid 328')
if P1_atom.n_atoms != 1 or P2_atom.n_atoms != 1 or P3_atom.n_atoms != 1:
    raise ValueError('Error: more than one atom selected')

lig_atom = u_merge.select_atoms(f'resname {u_lig.atoms.resnames[0]} and name C12')

# get ll_x,y,z distances
r_vect = lig_atom.center_of_mass() - P1_atom.positions
print(f'l1_x: {r_vect[0][0]:.2f}')
print(f'l1_y: {r_vect[0][1]:.2f}')
print(f'l1_z: {r_vect[0][2]:.2f}')

l1_x: 1.98
l1_y: -6.75
l1_z: 4.06


Modify the `test/abfe/abfe.in` file to include the following lines based on the output of the above script:
    
```bash
    l1_x: xxx
    l1_y: xxx
    l1_z: xxx
```

In [10]:
sim_config = get_configure_from_file('data/input_files/abfe.in')

In [12]:
# Prepare the system for the equilibration stage
system.prepare(
    stage='equil',
#    input_file='data/md_input/lipid_abfe.in',
    input_file=sim_config,
    overwrite=True,
)

INFO | Simulation configuration: software='amber' calc_type='dock' celpp_receptor='7T2G' poses_list=['0'] p1=':149@CA' p2=':119@CA' p3=':328@CA' ligand_list=[] other_mol=[] solv_shell=4.0 lipid_mol=['POPC', 'PA', 'PC', 'OL'] fe_type='express' components=['m', 'n', 'e', 'v'] release_eq=[10.0, 2.5, 0.5, 0.0] attach_rest=[0.0, 0.1, 0.24, 0.56, 1.33, 3.16, 7.5, 10.5, 17.78, 20.0, 42.17, 50.0, 60.0, 75.0, 80.0, 100.0] ti_points=0 lambdas=[0.0001, 0.02, 0.04, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.92, 0.96, 0.98, 0.9999] sdr_dist=40.0 dec_method='sdr' dec_int='mbar' blocks=5 rec_dihcf_force=50.0 rec_discf_force=5.0 lig_distance_force=5.0 lig_angle_force=250.0 lig_dihcf_force=70.0 rec_com_force=10.0 lig_com_force=10.0 water_model='TIP3P' num_waters=0 buffer_x=12.0 buffer_y=12.0 buffer_z=22.0 lig_buffer=15.0 neutralize_only='no' cation='Na+' anion='Cl-' ion_conc=0.15 hmr='yes' temperature=310.0 eq_steps1=100000 eq_steps2=5000000 n_steps_

In [14]:
# Submit the equilibration job to SLURM
system.submit(stage='equil')

INFO | Submit equilibration stage
INFO | Equilibration systems have been submitted for all poses listed in the input file.


Wait for the equilbration to finish.

In [ ]:
# Prepare the system for the free energy calculation
system.prepare(
    stage='fe',
#    input_file='data/md_input/lipid_abfe.in',
    input_file=sim_config,
#    overwrite=True,
)

In [ ]:
# Submit the equilibration job to SLURM
system.submit(stage='fe')